In [68]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from sklearn.neighbors import NearestCentroid

import time

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import clustering_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.util.tf_export import estimator_export
from tensorflow_estimator.python.estimator import estimator
from tensorflow_estimator.python.estimator import model_fn as model_fn_lib
from tensorflow_estimator.python.estimator.export import export_output

In [82]:
points = pd.read_csv('/content/Data.csv')

def input_fn():
  return tf.compat.v1.train.limit_epochs(
      tf.convert_to_tensor(points, dtype=tf.float32), num_epochs=1)

num_clusters = 3
kmeans = tf.compat.v1.estimator.experimental.KMeans(
    num_clusters=num_clusters, use_mini_batch=False)

# train
num_iterations = 10
previous_centers = None
for _ in range(num_iterations):
  kmeans.train(input_fn)
  cluster_centers = kmeans.cluster_centers()
  if previous_centers is not None:
    print('delta:', cluster_centers - previous_centers)
  previous_centers = cluster_centers
  print('score:', kmeans.score(input_fn))
print('cluster centers:', cluster_centers)

# map the input points to their clusters
cluster_indices = list(kmeans.predict_cluster_index(input_fn))
for i, point in enumerate(points):
  cluster_index = cluster_indices[i]
  center = cluster_centers[cluster_index]
  print('point:', point, 'is in cluster', cluster_index, 'centered at', center)
kmeans.cluster_centers()


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_aoivm3u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten

array([[3.4731708, 2.7560976, 3.3463414, 2.9609756, 3.5951219, 3.4341464,
        2.9707317, 3.6292684, 3.5414634, 3.5609756, 3.5804877, 3.5317073,
        3.7707317, 3.1414635, 2.814634 , 3.497561 , 3.702439 , 3.2146342,
        2.8536584, 3.4048781, 3.507317 , 3.385366 , 2.4195123, 3.4585366,
        3.1317074, 3.692683 , 3.4243903, 3.4731708, 3.5609756, 3.2195122,
        3.4195123, 3.2439024, 3.5365853, 3.687805 , 3.0536585, 3.2487805,
        3.4536586, 3.5414634, 3.5658536, 3.6      , 2.9414635, 3.7170732],
       [1.7212644, 1.4137931, 1.4252874, 1.2787356, 1.841954 , 1.7097701,
        1.2442529, 1.7557471, 1.7155173, 1.4971265, 1.7873564, 1.6321839,
        1.7356322, 1.7931035, 1.2557471, 1.6408046, 1.5545977, 1.7097701,
        1.3534483, 1.3821839, 1.3764368, 1.7155173, 1.1695403, 1.6350574,
        1.5114943, 1.7413793, 1.8189656, 1.3649426, 1.6982758, 1.4683908,
        1.5977012, 1.7988505, 1.6954023, 1.5      , 1.6781609, 1.2758621,
        1.4137931, 1.4568965, 1.84770

In [90]:
X = kmeans.cluster_centers()
y = np.array(["Moderate","Mild","Sevre"])
clf = NearestCentroid()
clf.fit(X, y)

print(clf.predict([points.loc[129]]))


['Sevre']
